In [10]:
!pip install backtrader
!pip install shioaji
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from matplotlib.dates import date2num
import backtrader as bt
from google.colab import userdata

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')

# 登入你的key
accounts = api.login(shioaji_apikey, shioaji_secret)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 2.7 MB/s eta 0:00:00
Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [3]:
# 日k的資料

def get_daily_kbar(stock_code, start_date, end_date):
  kbars = api.kbars(
    contract=api.Contracts.Stocks[stock_code],
    start=start_date,
    end=end_date,
  )
  df = pd.DataFrame({**kbars})
  df.ts = pd.to_datetime(df.ts)

  # 依照日期分組，並計算每組的Open, Close, High, Low
  daily_kbar = df.resample('D', on='ts').agg({
      'Open': 'first',
      'Close': 'last',
      'High': 'max',
      'Low': 'min',
      'Volume': 'sum'  # 計算每日成交量的總和
  }).dropna()  # 移除空白的天數
  # 計算當日漲跌幅
  # 取得前一天的Close價
  daily_kbar['Previous_Close'] = daily_kbar['Close'].shift(1)

  # 計算今天的漲跌幅 (今天的Close - 昨天的Close) / 昨天的Close * 100
  daily_kbar['Change_Percentage'] = ((daily_kbar['Close'] - daily_kbar['Previous_Close']) / daily_kbar['Previous_Close'] * 100).round(2)

  # 計算當日震幅
  daily_kbar['Amplitude'] = ((daily_kbar['High'] - daily_kbar['Low']) / daily_kbar['High'] * 100).round(2)

  # 移除前一天沒有數據的行
  daily_kbar = daily_kbar.dropna(subset=['Previous_Close'])
  print(daily_kbar)
  # 將結果輸出到Excel檔案
  #daily_kbar.to_excel(f'{stock_code}-daily_kbar.xlsx', index=True)  # `index=True` 會將日期作為列索引輸出
  return daily_kbar


In [5]:
start_day ="2020-01-01"
end_day = "2020-12-31"

# goodStockList2020=[4743,6443,6531,4128,3545 ,6547,2609 ,8046 ,3218 ,6492 ,2368 ,2603 ,4961 ,2303 ,2615 ,3661 ,5269 ,4142 ,6415, 3374]
# for(stock_code)in goodStockList2020:
#     stock_code = str(stock_code)
#     get_daily_kbar(stock_code,start_day,end_day)

# get_daily_kbar(f"{stock_code}",start_day,end_day)

df_2609 = get_daily_kbar("2609",start_day,end_day)

             Open  Close   High    Low  Volume  Previous_Close  \
ts                                                               
2020-01-03   7.36   7.32   7.36   7.26    1637            7.28   
2020-01-06   7.34   7.26   7.36   7.26    1368            7.32   
2020-01-07   7.28   7.28   7.30   7.26     948            7.26   
2020-01-08   7.28   7.20   7.28   7.20     761            7.28   
2020-01-09   7.21   7.22   7.29   7.21     468            7.20   
...           ...    ...    ...    ...     ...             ...   
2020-12-25  23.10  23.10  23.10  22.35  180034           21.00   
2020-12-28  24.30  25.40  25.40  24.30   83241           23.10   
2020-12-29  27.90  27.90  27.90  25.40  315516           25.40   
2020-12-30  27.30  27.50  28.05  25.65  110869           27.90   
2020-12-31  27.50  29.25  29.25  27.25   99887           27.50   

            Change_Percentage  Amplitude  
ts                                        
2020-01-03               0.55       1.36  
2020-01-06  

In [7]:
# sma cross strategy
class SmaCross(bt.Strategy):
    # 交易紀錄
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    # 設定交易參數
    params = dict(
        ma_period_short=5,
        ma_period_long=10
    )

    def __init__(self):
        # 均線交叉策略
        sma1 = bt.ind.SMA(period=self.p.ma_period_short)
        sma2 = bt.ind.SMA(period=self.p.ma_period_long)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

        # 使用自訂的sizer函數，將帳上的錢all-in
        self.setsizer(sizer())

        # 用開盤價做交易
        self.dataopen = self.datas[0].Close

    def next(self):
        # 帳戶沒有部位
        if not self.position:
            # 5ma往上穿越20ma
            if self.crossover > 0:
                # 印出買賣日期與價位
                self.log('BUY ' + ', Price: ' + str(self.dataopen[0]))
                # 使用開盤價買入標的
                self.buy(price=self.dataopen[0])
        # 5ma往下穿越20ma
        elif self.crossover < 0:
            # 印出買賣日期與價位
            self.log('SELL ' + ', Price: ' + str(self.dataopen[0]))
            # 使用開盤價賣出標的
            self.close(price=self.dataopen[0])

# 計算交易部位
class sizer(bt.Sizer):
    def _getsizing(self, comminfo, cash, data, isbuy):
        if isbuy:
            return math.floor(cash/data[1])
        else:
            return self.broker.getposition(data)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1330', ...)

In [9]:
print(output)

Start                     2020-01-03 00:00:00
End                       2020-12-31 00:00:00
Duration                    363 days 00:00:00
Exposure Time [%]                   87.704918
Equity Final [$]                  40535.84746
Equity Peak [$]                   41125.44746
Return [%]                         305.358475
Buy & Hold Return [%]              299.590164
Return (Ann.) [%]                  324.393196
Volatility (Ann.) [%]              240.180616
Sharpe Ratio                         1.350622
Sortino Ratio                       12.216155
Calmar Ratio                         15.43633
Max. Drawdown [%]                  -21.014917
Avg. Drawdown [%]                    -6.34575
Max. Drawdown Duration       58 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      171.73376
Worst Trade [%]                    -15.207723
Avg. Trade [%]                    